In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import csv
import requests
from config import api_key_omdb2
import time

In [2]:
'''
#Load in csv with IMBD ids found on Kaggle
#(https://www.kaggle.com/rounakbanik/the-movies-dataset#links.csv)
omdb_link_file = pd.read_csv('.\\raw_data\\links.csv')
omdb_link_data = pd.DataFrame(omdb_link_file)

#Add leading zeros and tt to the IMBD id column
omdb_link_data['imdbId'] = omdb_link_data['imdbId'].apply(lambda x: 'tt{0:0>7}'.format(x))

#Convert omdb column to list
omdb_id_list = omdb_link_data['imdbId'].tolist()

len(omdb_id_list)
'''

45843

In [6]:
'''
API call, only need to run once
url = "http://private.omdbapi.com/"
params = {"apikey":api_key_omdb2}
not_found = []

counter = 0
start_time = time.time()
for x in range (len(omdb_id_list)):

    movieid = omdb_id_list[x]
    params["i"] = movieid
    
    response = requests.get(url,params=params).json()
    try:
        # assemble url and make API request
        omdb_link_data.loc[x,'Title'] = response.get("Title",'')
        omdb_link_data.loc[x,'Year'] = response.get("Year",'')
        omdb_link_data.loc[x,'Released'] = response.get("Released",'')
        omdb_link_data.loc[x,'Runtime'] = response.get("Runtime",'')
        omdb_link_data.loc[x,'Genre'] = response.get("Genre",'')
        omdb_link_data.loc[x,'Director'] = response.get("Director",'')
        omdb_link_data.loc[x,'Writer'] = response.get("Writer",'')
        omdb_link_data.loc[x,'Actors'] = response.get("Actors",'')
        omdb_link_data.loc[x,'Plot'] = response.get("Plot",'')
        omdb_link_data.loc[x,'Country'] = response.get("Country",'')
        omdb_link_data.loc[x,'Awards'] = response.get("Awards",'')
        omdb_link_data.loc[x,'RT_Ratings'] = response["Ratings"][1].get("Value",'')
        omdb_link_data.loc[x,'Metascore'] = response.get("Metascore",'')
        omdb_link_data.loc[x,'imdbRating'] = response.get("imdbRating",'')
        omdb_link_data.loc[x,'imdbVotes'] = response.get("imdbVotes",'')
        omdb_link_data.loc[x,'imdbID'] = response.get("imdbID",'')
        omdb_link_data.loc[x,'Type'] = response.get("Type",'')
        omdb_link_data.loc[x,'BoxOffice'] = response.get("BoxOffice",'')
        omdb_link_data.loc[x,'Production'] = response.get("Production",'')
        
    except (KeyError, ValueError) as e:
        print(f"skipping {movieid}")
        not_found.append(movieid)
    except IndexError:
        omdb_link_data.loc[x,'Title'] = response.get("Title",'')
        omdb_link_data.loc[x,'Year'] = response.get("Year",'')
        omdb_link_data.loc[x,'Released'] = response.get("Released",'')
        omdb_link_data.loc[x,'Runtime'] = response.get("Runtime",'')
        omdb_link_data.loc[x,'Genre'] = response.get("Genre",'')
        omdb_link_data.loc[x,'Director'] = response.get("Director",'')
        omdb_link_data.loc[x,'Writer'] = response.get("Writer",'')
        omdb_link_data.loc[x,'Actors'] = response.get("Actors",'')
        omdb_link_data.loc[x,'Plot'] = response.get("Plot",'')
        omdb_link_data.loc[x,'Country'] = response.get("Country",'')
        omdb_link_data.loc[x,'Awards'] = response.get("Awards",'')
        omdb_link_data.loc[x,'Metascore'] = response.get("Metascore",'')
        omdb_link_data.loc[x,'imdbRating'] = response.get("imdbRating",'')
        omdb_link_data.loc[x,'imdbVotes'] = response.get("imdbVotes",'')
        omdb_link_data.loc[x,'imdbID'] = response.get("imdbID",'')
        omdb_link_data.loc[x,'Type'] = response.get("Type",'')
        omdb_link_data.loc[x,'BoxOffice'] = response.get("BoxOffice",'')
        omdb_link_data.loc[x,'Production'] = response.get("Production",'')
            
end_time = time.time()
elapsed = end_time - start_time
print(f"{elapsed} seconds elapsed, {len(not_found)} record(s) not found")
omdb_link_data.to_csv('.\\raw_data\\raw_omdb.csv', encoding="utf-8", index=False)
'''

skipping tt1628842
7644.897067070007 seconds elapsed, 1 record(s) not found


In [2]:
omdb_file = pd.read_csv('.\\raw_data\\raw_omdb.csv')
omdb_data = pd.DataFrame(omdb_file)

In [3]:
omdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 22 columns):
movieId       45843 non-null int64
imdbId        45843 non-null object
tmdbId        45624 non-null float64
Title         45842 non-null object
Year          45842 non-null object
Released      44060 non-null object
Runtime       45338 non-null object
Genre         45650 non-null object
Director      45254 non-null object
Writer        42894 non-null object
Actors        44998 non-null object
Plot          44213 non-null object
Country       45728 non-null object
Awards        25722 non-null object
RT_Ratings    19457 non-null object
Metascore     11586 non-null float64
imdbRating    45712 non-null float64
imdbVotes     45711 non-null object
imdbID        45842 non-null object
Type          45842 non-null object
BoxOffice     5433 non-null object
Production    30586 non-null object
dtypes: float64(3), int64(1), object(18)
memory usage: 7.7+ MB


In [4]:
def ratings_parse(x):
    if '%' in x:
        return x.split('%')[0]
    elif '/' in x:
        return x.split('/')[0]
    elif x is None:
        return None

In [5]:
#Filter out any series or episode
omdb_movie = omdb_data[omdb_data["Type"]=='movie']

In [6]:
#Filter out titles that contain #DUPE#
omdb_movies = omdb_movie[omdb_movie.Title != '#DUPE#']

In [7]:
#Retain only needed column for movie analysis
mod_omdb_movie_df = omdb_movies[['Title','Year','Genre','Country','RT_Ratings','Metascore','imdbRating','imdbVotes','Type','Production']]

In [8]:
#Filter out movies from before 1970
mod_omdb_movie_df= mod_omdb_movie_df[mod_omdb_movie_df["Year"] >= '1970']

In [9]:
mod_omdb_movie_rt_ratings= mod_omdb_movie_df[mod_omdb_movie_df["RT_Ratings"].notnull()]

In [10]:
#Normalize Rotten Tomatoes ratings
mod_omdb_movie_rt_ratings["RT_Ratings2"] = mod_omdb_movie_rt_ratings["RT_Ratings"].apply(ratings_parse)

C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
#Normalize imdb Ratings
mod_omdb_movie_rt_ratings["imdbRating2"] = mod_omdb_movie_rt_ratings["imdbRating"].multiply(10)

C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
#Save as movies CSV
mod_omdb_movie_rt_ratings.to_csv('.\\cleaned_data\\omdb_movies_cleaned.csv', encoding="utf-8", index=False)